# 导入相关库

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# 导入数据

In [ ]:
df = pd.read_csv(r'../input/income-classification/income_evaluation.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe(include='all')

变量： 'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship','race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'                                      
这些数据是1994年人口普查局数据库中提取的。通过使用这个数据集，我尝试预测这个人的收入是超过5万还是低于5万，我用python实现了随机森林分类。另外，我还使用了集成学习来提高模型的精度。

# 数据预处理

### 纠正的列名称

In [ ]:
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship','race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

### 检查缺失值

In [ ]:
df.isna().sum()

没有缺失值

#### 检查变量的数据类型

In [ ]:
df.dtypes

### 分离分类和数值变量

In [ ]:
num = [i for i in df.columns if df[i].dtype!='O']
cat = [i for i in df.columns if df[i].dtype=='O']

### EDA分类变量

In [ ]:
df[cat].nunique().plot(kind='bar')

检查每个分类的不同类别变量

In [ ]:
for i in df[cat]:
    print(df[i].value_counts())

有一些变量中存在'？'值而不是空值。我们需要删除这些值。

### ****将所有无效值替换为NaN值****

In [ ]:
df.replace(' ?', np.nan, inplace= True)

See we can easily get the count of NaN values

In [ ]:
df.isna().sum()

##### 在这里可以看到变量中的空值计数

### 使用fillna（）函数替换前一列的空值

In [ ]:
df.fillna(method = 'bfill', inplace=True)

In [ ]:
df.isna().sum().sum()

# 察看数据分布

In [ ]:
sea.pairplot(df)

# 收入与其他变量间的关系

In [ ]:
df.income.value_counts()

In [ ]:
sea.countplot(x= 'income' ,data =df)

In [ ]:
df.income.value_counts().plot(kind='pie')

有超过75%的人数收入不到50k

### 与性别

In [ ]:
sea.countplot(x="income", hue="sex", data=df)

两个维度上都是男性高于女性

### 与工作类型

In [ ]:
plt.subplots(figsize=(12, 8))
sea.countplot(x="income", hue="workclass", data=df)

两个维度上都是私营的从业人员最多

### 工作类型和性别的关系

In [ ]:
plt.subplots(figsize=(12, 8))
sea.countplot(hue="sex", x="workclass", data=df)

在工作类型中最多的是私营，所有工作类型中都是男性最多

In [ ]:
for i in cat:
    print(i, ' contains ', len(df[i].unique()), ' labels')

### 对分类变量进行编码

#### 删除目标收入

In [ ]:
y = df.income

In [ ]:
df

In [ ]:
y = pd.get_dummies(y,drop_first=True)
df.drop(['income'],axis =1, inplace=True)
num1 = [i for i in df.columns if df[i].dtype!='O']
cat1 = [i for i in df.columns if df[i].dtype=='O']

In [ ]:
x = pd.get_dummies(df[cat1])

In [ ]:
df.drop(df[cat1],axis = 1,inplace = True)

# EDA 数值变量

In [ ]:
df[num1].head()

#### 查看年龄变量的分布

In [ ]:
sea.distplot(df.age, bins=10)

大多数人属于20至50岁年龄段。

### 检查数值变量中的异常值

In [ ]:
sea.boxplot(df.age)

Age变量中存在一些异常值

### 检查数值变量之间的相关性

In [ ]:
df.corr()

变量之间没有相关性

### 缩放数值变量

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df = scaler.fit_transform(df)

In [ ]:
df = pd.DataFrame(df)
x = pd.concat([x,df],axis=1)

### 将数据分为测试和训练

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

### 可视化训练数据

In [ ]:
X_train.head()

### 逻辑回归模型

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

###  通过预测测试数据并计算准确性得分来测试逻辑回归模型

In [ ]:
y_pred = logreg.predict(X_test)
accuracy_score(y_test, y_pred)

### 10个决策树的随机森林模型

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0,n_estimators=10)
rfc.fit(X_train, y_train)

### 通过预测测试数据并计算准确性得分来测试随机森林模型

In [ ]:
y_pred = rfc.predict(X_test)
accuracy_score(y_test, y_pred)

### 用100个决策树拟合随机森林模型

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0,n_estimators=100)
rfc.fit(X_train, y_train)

### 通过预测测试数据来测试随机森林模型，并使用100个决策树计算准确性得分

In [ ]:
y_pred = rfc.predict(X_test)
accuracy_score(y_test, y_pred)

##### 通过增加决策树的数量，准确性提高了0.1

## 使用Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(random_state=0)
model.fit(X_train, y_train)

### 测试bagging model

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

# 保持超参数（学习率为0.1）拟合极端梯度提升方法，然后计算精度

In [ ]:
import xgboost as xgb
model=xgb.XGBClassifier(base_estimator = rfc,random_state=1,learning_rate=0.1)
model.fit(X_train, y_train)
model.score(X_test,y_test)


**摘要**
- 本次收入预测分类问题，logistic回归达到85.13%。
- 无基模型的bagging准确率为84.39%
- 基于10决策树基模型随机林的集成学习bagging，准确率为84.95%
- 基于100个决策树的基模型随机林集成学习bagging，准确率为85.40%
- 基于随机森林的集成学习boosting技术，准确率为86.94%